In [ ]:
# Импорты
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Загрузка данных
df = pd.read_csv('ecommerce_data.csv')
df['date'] = pd.to_datetime(df['date'])
df.isnull().sum()

In [ ]:
# Агрегация по дням
data_orders = df.groupby('date')['orders'].sum().reset_index()

In [ ]:
# Фичи из даты
data_orders['dayofweek'] = data_orders['date'].dt.dayofweek
data_orders['day'] = data_orders['date'].dt.day
data_orders['month'] = data_orders['date'].dt.month
data_orders['is_weekend'] = data_orders['dayofweek'].isin([5,6]).astype(int)

In [ ]:
# Лаги и скользящие
for lag in [1,7,14]:
    data_orders[f'lag_{lag}'] = data_orders['orders'].shift(lag)
for win in [3,7,14]:
    data_orders[f'rolling_mean_{win}'] = data_orders['orders'].shift(1).rolling(window=win).mean()

In [ ]:
# Добавление признаков начала/конца месяца
data_orders['is_month_start'] = data_orders['date'].dt.is_month_start.astype(int)
data_orders['is_month_end'] = data_orders['date'].dt.is_month_end.astype(int)

In [ ]:
# Предобработка
data_orders = data_orders.dropna().copy()
data_orders['log_orders'] = np.log1p(data_orders['orders'])

In [ ]:
# Разделение на train/test
split_date = data_orders['date'].max() - pd.Timedelta(days=30)
train = data_orders[data_orders['date'] < split_date]
test = data_orders[data_orders['date'] >= split_date]

In [ ]:
# LightGBM обучение
from lightgbm import LGBMRegressor
features = [col for col in data_orders.columns if col not in ['date', 'orders', 'log_orders']]
X_train, y_train = train[features], train['log_orders']
X_test, y_test = test[features], test['log_orders']
model = LGBMRegressor(n_estimators=1000, learning_rate=0.1)
model.fit(X_train, y_train)

In [ ]:
# Предсказание и обратное преобразование
y_pred = model.predict(X_test)
y_pred = np.expm1(y_pred)
y_test = np.expm1(y_test)

In [ ]:
# MAPE и визуализация
mape = np.mean(np.abs((y_test - y_pred) / y_test))
print(f'MAPE модели LightGBM: {mape:.2%}')

plt.figure(figsize=(14,4))
plt.plot(test['date'], y_test, label='Факт')
plt.plot(test['date'], y_pred, label='Прогноз')
plt.legend()
plt.grid(True)
plt.title('LightGBM: прогноз vs факт')
plt.tight_layout()
plt.show()